In [47]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import max_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
import xgboost as xgb

In [17]:
df = pd.read_csv('feature_eng.csv')

In [18]:
df = df.drop(columns=['Unnamed: 0'])

In [19]:
df.isnull().sum()

fl_date                 0
origin                  0
dest                    0
distance                0
taxi_time               0
actual_elapsed_time     0
origin_delay_airport    0
dest_delay_airport      0
arr_delay               0
origin_delays           0
origin_delay            0
dest_delay              0
delay_by_month          0
delay_by_day            0
dtype: int64

In [14]:
#data split 30% 70%
def split_data(df, drop_list):
    df = df.drop(drop_list,axis=1)
    print(df.columns)
    #test train split time
    from sklearn.model_selection import train_test_split
    y = df['arr_delay'].values #target
    X = df.drop(['arr_delay'],axis=1).values #features
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=100)

    print("train-set size: ", len(y_train),
      "\ntest-set size: ", len(y_test))
    return X_train, X_test, y_train, y_test

In [15]:
def get_predictions(clf, X_train, y_train, X_test):
    # create classifier
    clf = clf
    # fit it to training data
    clf.fit(X_train,y_train)
    # predict using test data
    y_pred = clf.predict(X_test)
    return y_pred

In [16]:
#Function to print the regressor's scores
def print_scores(y_test,y_pred):
    print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))
    print('R2: ', r2_score(y_test, y_pred))
    print('Max Error: ', max_error(y_test, y_pred))

In [51]:
df

,fl_date,origin,dest,distance,taxi_time,actual_elapsed_time,origin_delay_airport,dest_delay_airport,arr_delay,origin_delays,origin_delay,dest_delay,delay_by_month,delay_by_day
0,2019-05-19,MCI,AUS,650.0,6.0,105.0,MCI,AUS,-9.0,0.813154,0.813154,1.996324,6.123189,6.025008
1,2018-04-07,PHL,MCO,861.0,-5.0,163.0,PHL,MCO,-7.0,5.809959,5.809959,7.487434,4.164554,4.668227
2,2018-01-01,DCA,CAE,408.0,10.0,87.0,DCA,CAE,53.0,5.816488,5.816488,8.679389,3.394647,4.816586
3,2018-01-07,SGF,IAH,513.0,4.0,107.0,SGF,IAH,1.0,11.879032,11.879032,8.660893,3.394647,4.668227
4,2019-05-08,SLC,PHX,507.0,14.0,93.0,SLC,PHX,-12.0,1.295573,1.295573,5.465170,6.123189,6.845741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98001,2018-03-14,TUS,MDW,1440.0,3.0,189.0,TUS,MDW,-10.0,0.395918,0.395918,2.467472,2.197540,5.106019
98002,2018-02-28,ALB,MCO,1073.0,7.0,159.0,ALB,MCO,-20.0,2.696429,2.696429,7.487434,6.120882,4.031586
98003,2018-03-17,LAS,CLT,1916.0,8.0,220.0,LAS,CLT,17.0,3.668244,3.668244,2.815619,2.197540,5.666029
98004,2018-09-09,ATL,LAX,1947.0,4.0,274.0,ATL,LAX,-10.0,2.446831,2.446831,2.834677,1.452672,6.438135


In [53]:
#Tree1
data1 = ['fl_date','origin','dest','distance','actual_elapsed_time','origin_delay_airport','dest_delay_airport','taxi_time','origin_delay','dest_delay','delay_by_month','delay_by_day']
X_train, X_test, y_train, y_test = split_data(df, data1)
y_pred = get_predictions(RandomForestRegressor(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['arr_delay', 'origin_delays'], dtype='object')
train-set size:  68604 
test-set size:  29402
Mean Absolute Error:  24.501180025496637
R2:  -0.007128889004163641
Max Error:  1197.229046921227


In [26]:
#Tree2 - took out calculated averages
data2 = ['fl_date','origin','dest','distance','actual_elapsed_time','origin_delay_airport','dest_delay_airport','taxi_time']
X_train, X_test, y_train, y_test = split_data(df, data2)
y_pred = get_predictions(RandomForestRegressor(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['arr_delay', 'origin_delays', 'origin_delay', 'dest_delay',
       'delay_by_month', 'delay_by_day'],
      dtype='object')
train-set size:  68604 
test-set size:  29402
Mean Absolute Error:  27.7817868257016
R2:  -0.16987028673113702
Max Error:  1167.68


In [43]:
#Tree3 - removed redundancy
data3 = ['origin','dest','distance','actual_elapsed_time','origin_delay_airport','dest_delay_airport']
X_train, X_test, y_train, y_test = split_data(df, data2)
y_pred = get_predictions(RandomForestRegressor(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['arr_delay', 'origin_delays', 'origin_delay', 'dest_delay',
       'delay_by_month', 'delay_by_day'],
      dtype='object')
train-set size:  68604 
test-set size:  29402
Mean Absolute Error:  27.764312896240135
R2:  -0.16595190888769173
Max Error:  1181.79


In [58]:
#LinearReg
X_train, X_test, y_train, y_test = split_data(df, data1)
y_pred = get_predictions(LinearRegression(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['arr_delay', 'origin_delays'], dtype='object')
train-set size:  68604 
test-set size:  29402
Mean Absolute Error:  24.365087054283006
R2:  0.0047212075254682295
Max Error:  1195.7473130023247


In [28]:
#Gaussian1
X_train, X_test, y_train, y_test = split_data(df, data1)
y_pred = get_predictions(GaussianNB(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['arr_delay', 'origin_delays'], dtype='object')
train-set size:  68604 
test-set size:  29402
Mean Absolute Error:  258.6155023467791
R2:  -47.37626548344507
Max Error:  1561.0


In [34]:
#Gaussian2
X_train, X_test, y_train, y_test = split_data(df, data2)
y_pred = get_predictions(GaussianNB(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['arr_delay', 'origin_delays', 'origin_delay', 'dest_delay',
       'delay_by_month', 'delay_by_day'],
      dtype='object')
train-set size:  68604 
test-set size:  29402
Mean Absolute Error:  55.90180940072104
R2:  -5.271023234136759
Max Error:  1210.0


In [ ]:
#Gaussian3 - Do not understand such weird error, but pretty bad results
X_train, X_test, y_train, y_test = split_data(df, data3)
y_pred = get_predictions(GaussianNB(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

In [40]:
# KNN1
X_train, X_test, y_train, y_test = split_data(df, data1)
y_pred = get_predictions(KNeighborsRegressor(n_neighbors=2000), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['arr_delay', 'origin_delays'], dtype='object')
train-set size:  68604 
test-set size:  29402
Mean Absolute Error:  24.327206199124777
R2:  0.0055436662994825525
Max Error:  1197.352


In [41]:
# KNN2
X_train, X_test, y_train, y_test = split_data(df, data2)
y_pred = get_predictions(KNeighborsRegressor(n_neighbors=2000), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['arr_delay', 'origin_delays', 'origin_delay', 'dest_delay',
       'delay_by_month', 'delay_by_day'],
      dtype='object')
train-set size:  68604 
test-set size:  29402
Mean Absolute Error:  24.006139242228418
R2:  0.01648696362742441
Max Error:  1188.0055


In [ ]:
# KNN3
X_train, X_test, y_train, y_test = split_data(df, data3)
y_pred = get_predictions(KNeighborsRegressor(n_neighbors=2000), X_train, y_train, X_test)
print_scores(y_test,y_pred)

In [56]:
#Ridge best result 
X_train, X_test, y_train, y_test = split_data(df, data1)
y_pred = get_predictions(Ridge(alpha=1), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['arr_delay', 'origin_delays'], dtype='object')
train-set size:  68604 
test-set size:  29402
Mean Absolute Error:  24.365086901731058
R2:  0.004721211039422135
Max Error:  1195.7473156055823


In [57]:
#xgboost 
X_train, X_test, y_train, y_test = split_data(df, data1)
y_pred = get_predictions(xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 3, alpha = 1, n_estimators = 20), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['arr_delay', 'origin_delays'], dtype='object')
train-set size:  68604 
test-set size:  29402
[19:39:46] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error:  24.101621574268446
R2:  0.0027928185228006663
Max Error:  1197.282401561737
